In [1]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# !tar xf spark-3.3.2-bin-hadoop3.tgz
# !mv spark-3.3.2-bin-hadoop3 spark
# !pip install -q findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark"

In [2]:
!pip install -q findspark

In [3]:
import findspark
findspark.init()

In [4]:
spark_url = 'local'

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Tutorial')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [6]:
from pyspark.sql import functions as F

In [7]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/df_with_cluster.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-------------+--------------------+--------------+----+------------+--------------------+---------+--------+----------+
|  ticket_id|                type|        organization|             comment|               photo|         photo_after|             address|subdistrict|district|     province|           timestamp|         state|star|count_reopen|       last_activity|longitude|latitude|cluster_id|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-------------+--------------------+--------------+----+------------+--------------------+---------+--------+----------+
|2021-FYJTFP|         {ความสะอา

In [8]:
# df = spark.read.csv("df_with_cluster.csv", header=True, inferSchema=True)

In [9]:
# df.show(3)

In [10]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType, StringType, DoubleType
from pyspark.sql import Row

In [11]:
# Drop row that contain "noise" in cluster_id
df = df.filter(~F.col("cluster_id").contains("noise"))

In [12]:
# # Drop row that contain "noise" in cluster_id
# df = df.filter(~F.col("cluster_id").contains("noise"))

# Cast to correct type
df = df.withColumn("timestamp_ts", F.col("timestamp").cast("timestamp")) \
       .withColumn("last_activity_ts", F.col("last_activity").cast("timestamp"))

# Sort before put in alogo
df_sorted = df.orderBy("last_activity_ts", "timestamp_ts")

# Drop unneccessary columns
col_to_drop = ["type", "organization", "comment", "photo", "photo_after", "address", "subdistrict", "district", "province", "timestamp", "star", "count_reopen", "last_activity", "longitude", "latitude", "row_id"]
df_sorted = df_sorted.drop(*col_to_drop)
df_sorted.show(15)


+-----------+---------+----------+--------------------+--------------------+
|  ticket_id|    state|cluster_id|        timestamp_ts|    last_activity_ts|
+-----------+---------+----------+--------------------+--------------------+
|2022-LEN8TT|เสร็จสิ้น|     106_1|2022-05-25 08:50:...|2022-05-25 08:52:...|
|2022-BLLZW9|เสร็จสิ้น|     112_0|2022-05-26 06:44:...|2022-05-26 06:50:...|
|2022-37F6UA|เสร็จสิ้น|   20272_0|2022-05-27 04:58:...|2022-05-27 10:55:...|
|2022-KU8VWY|เสร็จสิ้น|     179_0|2022-05-29 10:14:...|2022-05-30 08:46:...|
|2022-LXEUG2|เสร็จสิ้น|     161_1|2022-05-29 09:27:...|2022-05-30 09:11:...|
|2022-GY8ECD|เสร็จสิ้น|     605_1|2022-05-31 03:52:...|2022-05-31 07:49:...|
|2022-HTA67R|เสร็จสิ้น|     154_0|2022-05-28 17:09:...|2022-05-31 08:20:...|
|2022-8CQG7A|เสร็จสิ้น|     727_2|2022-05-31 09:03:...|2022-05-31 12:18:...|
|2022-HX4Y8V|เสร็จสิ้น|     726_0|2022-05-31 09:01:...|2022-05-31 16:04:...|
|2022-7LPM9N|เสร็จสิ้น|     166_0|2022-05-29 09:47:...|2022-05-31 16:13:...|

In [13]:
df_sorted.show(100)

+-----------+---------+----------+--------------------+--------------------+
|  ticket_id|    state|cluster_id|        timestamp_ts|    last_activity_ts|
+-----------+---------+----------+--------------------+--------------------+
|2022-LEN8TT|เสร็จสิ้น|     106_1|2022-05-25 08:50:...|2022-05-25 08:52:...|
|2022-BLLZW9|เสร็จสิ้น|     112_0|2022-05-26 06:44:...|2022-05-26 06:50:...|
|2022-37F6UA|เสร็จสิ้น|   20272_0|2022-05-27 04:58:...|2022-05-27 10:55:...|
|2022-KU8VWY|เสร็จสิ้น|     179_0|2022-05-29 10:14:...|2022-05-30 08:46:...|
|2022-LXEUG2|เสร็จสิ้น|     161_1|2022-05-29 09:27:...|2022-05-30 09:11:...|
|2022-GY8ECD|เสร็จสิ้น|     605_1|2022-05-31 03:52:...|2022-05-31 07:49:...|
|2022-HTA67R|เสร็จสิ้น|     154_0|2022-05-28 17:09:...|2022-05-31 08:20:...|
|2022-8CQG7A|เสร็จสิ้น|     727_2|2022-05-31 09:03:...|2022-05-31 12:18:...|
|2022-HX4Y8V|เสร็จสิ้น|     726_0|2022-05-31 09:01:...|2022-05-31 16:04:...|
|2022-7LPM9N|เสร็จสิ้น|     166_0|2022-05-29 09:47:...|2022-05-31 16:13:...|

In [14]:
# from pyspark.sql.functions import col
# null_count = df.filter(col("state").isNull()).count()
# print(null_count)

# null_count_ts = df.filter(col("timestamp_ts").isNull()).count()
# print(null_count_ts)

In [15]:
grouped_rdd = df_sorted.rdd.groupBy(lambda row: row.cluster_id)

def count_problems(rows):
    rows = sorted(rows, key=lambda r: (r.timestamp_ts, r.last_activity_ts)) # Sort again to make sure
    count = 0
    current_end = None

    for row in rows:
        if current_end is None or row.timestamp_ts > current_end:
            count += 1
            current_end = row.last_activity_ts
        # else: overlapping, same problem

    return Row(cluster_id=str(rows[0].cluster_id), num_problems=count, state = rows[-1].state)

result_rdd = grouped_rdd.map(lambda kv: count_problems(kv[1]))

result_schema = StructType([
    StructField("cluster_id", StringType(), True),
    StructField("num_times", IntegerType(), True),
    StructField("status", StringType(), True),
])

result_df = spark.createDataFrame(result_rdd, schema=result_schema)

result_df.show(100)

+----------+---------+--------------+
|cluster_id|num_times|        status|
+----------+---------+--------------+
|     106_1|        1|     เสร็จสิ้น|
|     112_0|        2|     เสร็จสิ้น|
|   20272_0|        2|     เสร็จสิ้น|
|     179_0|        3|     เสร็จสิ้น|
|     161_1|        2|กำลังดำเนินการ|
|     605_1|       15|     เสร็จสิ้น|
|     154_0|       10|     เสร็จสิ้น|
|     727_2|        1|     เสร็จสิ้น|
|     726_0|        2|     เสร็จสิ้น|
|     166_0|        5|     เสร็จสิ้น|
|     204_1|       28|     เสร็จสิ้น|
|      12_1|        2|     เสร็จสิ้น|
|   26355_0|        3|     เสร็จสิ้น|
|     516_2|        1|     เสร็จสิ้น|
|     274_1|        1|     เสร็จสิ้น|
|    1070_0|        3|     เสร็จสิ้น|
|     373_1|        1|     เสร็จสิ้น|
|     847_0|        9|     เสร็จสิ้น|
|     845_0|        3|กำลังดำเนินการ|
|    1391_2|        2|     เสร็จสิ้น|
|    1483_5|        1|     เสร็จสิ้น|
|    1377_0|        2|   รอรับเรื่อง|
|    1369_1|        2|     เสร็จสิ้น|
|    1316_0|

In [16]:
output_path = '/content/drive/MyDrive/Colab Notebooks/result_df_output'


In [17]:
result_df.coalesce(1).write.option("header", True).mode("overwrite").csv(output_path)
